In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|export
# Python native modules
import os
from datetime import datetime
import os
import shutil
import json
# Third party libs
from fastcore.all import *
from nbdev.config import get_config
import yaml
from IPython.display import display, Markdown
from fastcore.all import call_parse
# Local modules

In [ ]:
#|default_exp nbdev_extensions

# Nbdev Extensions
> Extensions on the nbdev library for conda yml creation and nightly build support

In [ ]:
%%bash
cat > test_settings.ini << EndOfMessage
[DEFAULT]
lib_name = fastrl_test
user = josiahls
branch = master
version = 0.0.1
min_python = 3.6
requirements = fastai>=2.0.0 moviepy
pip_requirements = pytest nvidia-ml-py3 dataclasses pandas pyyaml
conda_requirements = jupyter notebook setuptools
dev_requirements = jupyterlab nbdev ipywidgets moviepy pygifsicle aquirdturtle_collapsible_headings
EndOfMessage

mkdir testing
cp test_settings.ini testing/test_settings.ini

mkdir: cannot create directory ‘testing’: File exists


In [ ]:
#|export
def dependencies(dev:bool=False,cfg_name='settings.ini'):
    "Gets a list of dependencies in a `cfg_name` for conda compatability."
    c = get_config(cfg_name)
    deps=[f'python={c.min_python}','pip','setuptools']
    if c.requirements:             deps+=c.requirements.split(' ')
    if c.conda_requirements:       deps+=c.conda_requirements.split(' ')
    if dev and c.dev_requirements: deps+=c.dev_requirements.split(' ')
    if c.pip_requirements:         deps+=[{'pip':c.pip_requirements.split(' ')}]
    return deps

In [ ]:
test_eq(dependencies(cfg_name='test_settings.ini'),
        ['python=3.6', 'pip', 'setuptools', 'fastai>=2.0.0', 'moviepy', 
         'jupyter', 'notebook', 'setuptools', 
         {'pip': ['pytest', 'nvidia-ml-py3', 'dataclasses', 'pandas', 'pyyaml']}])
test_eq(dependencies(dev=True,cfg_name='test_settings.ini'),
        ['python=3.6', 'pip', 'setuptools', 'fastai>=2.0.0', 'moviepy', 'jupyter', 
         'notebook', 'setuptools', 'jupyterlab', 'nbdev', 'ipywidgets', 'moviepy', 
         'pygifsicle', 'aquirdturtle_collapsible_headings', 
         {'pip': ['pytest', 'nvidia-ml-py3', 'dataclasses', 'pandas', 'pyyaml']}])

In [ ]:
#|export
def create_conda_yaml(channels:str='conda-forge,pytorch,fastai',
                      cfg_name='settings.ini',dev:bool=False):
    "Creates a conda dictionary of the format of an env file."
    c = get_config(cfg_name)
    return {'name':c.lib_name if not dev else c.lib_name+'_dev',
            'channels': channels.split(','),
            'dependencies': dependencies(dev=dev,cfg_name=cfg_name)}

In [ ]:
test_eq(create_conda_yaml(cfg_name='test_settings.ini'),
       {'name': 'fastrl_test', 'channels': ['conda-forge', 'pytorch', 'fastai'], 
        'dependencies': ['python=3.6', 'pip', 'setuptools', 'fastai>=2.0.0', 
                         'moviepy', 'jupyter', 'notebook', 'setuptools', 
                         {'pip': ['pytest', 'nvidia-ml-py3', 'dataclasses', 'pandas', 'pyyaml']}]})
test_eq(create_conda_yaml(cfg_name='test_settings.ini',dev=True),
       {'name': 'fastrl_test_dev', 'channels': ['conda-forge', 'pytorch', 'fastai'], 
        'dependencies': ['python=3.6', 'pip', 'setuptools', 'fastai>=2.0.0', 
                         'moviepy', 'jupyter', 'notebook', 'setuptools', 'jupyterlab', 
                         'nbdev', 'ipywidgets', 'moviepy', 'pygifsicle', 'aquirdturtle_collapsible_headings', 
                         {'pip': ['pytest', 'nvidia-ml-py3', 'dataclasses', 'pandas', 'pyyaml']}]})

In [ ]:
#|export
def create_conda_yamls(also_dev:bool=True,cfg_name='settings.ini',sub_dir=''):
    "Creates conda env for normal and development environments."
    c = get_config(cfg_name)
    parent=c.config_path/Path(sub_dir) if sub_dir else c.config_path
    parent.mkdir(parents=True,exist_ok=True)
    for is_dev in ([False,True] if also_dev else [False]):
        fname=(c.lib_name+f'{"_dev" if is_dev else ""}_env.yaml')
        with open(parent/fname,'w') as f:
            d=create_conda_yaml(cfg_name=cfg_name,dev=is_dev)
            yaml.dump(d,f)

In [ ]:
create_conda_yamls(cfg_name='test_settings.ini',sub_dir='testing')

In [ ]:
%pfile testing/fastrl_test_env.yaml

Object `testing/fastrl_test_env.yaml` not found.


channels:
- conda-forge
- pytorch
- fastai
dependencies:
- python=3.6
- pip
- setuptools
- fastai>=2.0.0
- moviepy
- jupyter
- notebook
- setuptools
- pip:
  - pytest
  - nvidia-ml-py3
  - dataclasses
  - pandas
  - pyyaml
name: fastrl_test


In [ ]:
create_conda_yamls(sub_dir='extra')

In [ ]:
%%bash
rm test_settings.ini

## Blogging Extensions

In [ ]:
#|export
def header(
    # The main header title to display.
    title: str,     
    # The subtitle to display underneath the title. If None, no subtitle will be displayed.                
    subtitle: Optional[str] = None, 
    # If True, the date associated with the header will be frozen, 
    # meaning it won't change in subsequent runs. 
    # If False, a new date will be generated each time the function is run,
    # and the date will not be saved to file.
    freeze: bool = False           
):
    """
    Function to generate a Markdown formatted header with an associated date.
    Dates are auto-incremented and can be frozen. This function also controls the persistent storage of dates.
    """
    filename = 'header_dates.json'
    date = None
    id:int = None

    # Load or initialize date dictionary
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            dates = json.load(file)
    else:
        dates = {}

    # Determine the id for the new entry
    if freeze:
        # If frozen, use the maximum id from the file, or 0 if the file is empty
        id = max(dates.keys(), default=0)
    else:
        # If not frozen, increment the maximum id from the file, or use 0 if the file is empty
        id = max(dates.keys(), default=-1) + 1

    # Get or create the date
    date = dates.get(id)
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
        dates[id] = date

        # Only write to file if the date is frozen
        if freeze:
            with open(filename, 'w') as file:
                json.dump(dates, file)

    # Display the markdown
    if subtitle is None:
        display(Markdown(f"# `{date}` **{title}**"))
    else:
        display(Markdown(f"# `{date}` **{title}**\n> {subtitle}"))


In [ ]:
header(1,"Some custom message",freeze=True)

# `2023-05-29` **1**
> Some custom message

In [ ]:
rm header_dates.json

In [ ]:
#|export
@call_parse
def create_blog_notebook() -> None: # Creates a new blog notebook from template
    template = '99_blog.from_xxxx_xx_to_xx.ipynb'
    new_name = datetime.now().strftime('99_blog.from_%Y_%m_to_now.ipynb')

    # Check if the template file exists
    if not os.path.exists(template):
        raise FileNotFoundError(f"Template file '{template}' not found in current directory.")

    # Rename old notebooks and update sidebar.yml
    sidebar_file = '../sidebar.yml'
    with open(sidebar_file, 'r') as f:
        sidebar = yaml.safe_load(f)

    blog_section = None
    for section in sidebar['website']['sidebar']['contents']:
        print(section)
        if 'section' in section and section['section'] == 'Blog':
            blog_section = section['contents']
            break

    # Rename old notebooks
    for filename in os.listdir():
        if filename.startswith('99_blog.from_') and filename.endswith('_to_now.ipynb'):
            date_from = filename[13:20]  # corrected substring indexing
            date_to = datetime.now().strftime('%Y_%m')
            new_filename = f'99_blog.from_{date_from}_to_{date_to}.ipynb'
            os.rename(filename, new_filename)

            if blog_section is not None:
                # Update sidebar.yml
                old_entry = f'12_Blog/{filename}'
                new_entry = f'12_Blog/{new_filename}'
                if old_entry in blog_section:
                    blog_section.remove(old_entry)
                    blog_section.append(new_entry)

    # Add new notebook to sidebar.yml
    if f'12_Blog/{new_name}' not in blog_section:
        blog_section.append(f'12_Blog/{new_name}')
        
        with open(sidebar_file, 'w') as f:
            yaml.safe_dump(sidebar, f)

    # Create new notebook from template
    shutil.copy(template, new_name)

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()

/usr/local/lib/python3.8/dist-packages/nbdev/export.py:54: UserWarning: Notebook '/home/fastrl_user/fastrl/nbs/07_Agents/02_Continuous/12u_agents.ppo.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
